In [1]:
%cd /code

/code


In [2]:
from source.service.datasets import DATA
from source.library.scraping import scrape_all_urls#, scrape_urls

# This is needed because openai.text_completion calls asynchronous functions but
# Jupyter is already running its own event loop.
import nest_asyncio
nest_asyncio.apply()

In [3]:
langchain_doc_chunks = DATA.langchain_doc_chunks.load()
print(len(langchain_doc_chunks))
print(langchain_doc_chunks[0])

1317
('https://python.langchain.com/en/deployments.html', '404 Not Found\n | Read the Docs Read the Docs 404 Not Found You are browsing the documentation of langchain . The documentation version you are looking for at /en/deployments.html was not found. Documentation changes over time and pages are moved around. You can try to navigate to the index page of the project and use its navigation, or search for a similar page. Try searching? Search all the docs')


In [4]:
from langchain.vectorstores import Chroma
persist_directory = '/code/.vectordb/'

In [5]:
vectordb = Chroma(persist_directory=persist_directory)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
vectordb.similarity_search(query="Document Loaders # Note Conceptual Guide Combining language models", k=5)

[Document(page_content='Document Loaders # Note Conceptual Guide Combining language models with your own text data is a powerful way to differentiate them.\nThe first step in doing this is to load the data into “documents” - a fancy way of say some pieces of text.\nThis module is aimed at making this easy. A primary driver of a lot of this is the Unstructured python package.\nThis package is a great way to transform all', metadata={'url': 'https://python.langchain.com/en/latest/modules/indexes/document_loaders.html'}),
 Document(page_content='Document Loaders\n==========================\n\n.. note::\n   `Conceptual Guide `_\n\n\nCombining language models with your own text data is a powerful way to differentiate them.\nThe first step in doing this is to load the data into "documents" - a fancy way of say some pieces of text.\nThis module is aimed at making this easy.\n\nA primary driver of a lot of this is the `Unstructured `_ python', metadata={'url': 'https://python.langchain.com/en/

In [7]:
# from langchain.chains import VectorDBQA
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp


model_path = './models/ggml-alpaca-7b-q4.bin'
llm = LlamaCpp(model_path=model_path, temperature=0.0, n_ctx=1024 * 2)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

llama.cpp: loading model from ./models/ggml-alpaca-7b-q4.bin
llama.cpp: can't use mmap because tensors are not aligned; convert to new format to avoid this
llama_model_load_internal: format     = 'ggml' (old version with low tokenizer quality and no mmap support)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size = 4113739.11 KB
llama_model_load_internal: mem required  = 5809.32 MB (+ 1026.00 MB per state)
...................................................................................................
.
A

---

In [8]:
question = "What is Langchain?"

In [9]:
llm(prompt=f"Question: {question}? Answer: ")

' Langchain is a distributed ledger technology that enables secure, private and reliable data sharing between organizations. It is based on blockchain technology but designed to meet the specific needs of enterprise customers.'

In [10]:
qa.run(question)

' Langchain is a programming language designed to make it easier for developers to create intelligent applications. It uses a declarative approach, allowing developers to specify what the application should do and let Langchain figure out how to do it. It has built-in machine learning capabilities, making it easy to develop AI-powered applications.'

---

In [11]:
question = "How do I load hugging face models with Langchain?"

In [12]:
llm(prompt=f"Question: {question}? Answer: ")

'1. Download the HuGG model from the GPT Model Hub. 2. Extract the files inside the zip folder and move them to a suitable location. 3. Open the Langchain CLI and type "load model <model_name>.<format>" where <model_name> is the name of the HuGG model you downloaded, and <format> is one of the following: "txt" (text), "bin" (binary) or "pbt" (pre-trained). 4. Follow the instructions in the Langchain CLI to load your model.'

In [13]:
answer = qa.run(question)
print(answer)

 You can use the Huggiing Face Hub wrapper to easily load models into Langchain. The wrapper is available as a library and can be installed with pip install huggiingface_hub. After installing the library, you will need to create an account on Huggiing Face Hub if you don't already have one. Once you have your model, you can use the Hueggiing Face Hub wrapper to load it into Langchain.


---

In [14]:
question = "How do I use prompt templates for creating a chat bot?"
print("Base-Model:")
print(llm(prompt=f"Question: {question}? Answer: "))
print("Context-Model")
context_answer = qa.run(question)
print(answer)

Base-Model:
1. Open the chatbot creator in Mitsuku. 2. Select the "Create a New Prompt" option from the main menu. 3. Choose a template from the list of available prompts. 4. Edit the template to customize it for your needs. 5. Preview and test the chatbot.
Context-Model
 You can use the Huggiing Face Hub wrapper to easily load models into Langchain. The wrapper is available as a library and can be installed with pip install huggiingface_hub. After installing the library, you will need to create an account on Huggiing Face Hub if you don't already have one. Once you have your model, you can use the Hueggiing Face Hub wrapper to load it into Langchain.


In [15]:
question = "How do I use prompt templates for creating a chat bot?"
print("Base-Model:")
base_answer = llm(prompt=f"Question: {question}? Answer: ")
print(base_answer)
print("Context-Model:")
context_answer = qa.run(question)
print(context_answer)

Base-Model:
1. Open the chatbot creator in Mitsuku. 2. Select the "Create a New Prompt" option from the main menu. 3. Choose a template from the list of available prompts. 4. Edit the template to customize it for your needs. 5. Preview and test the chatbot.
Context-Model:
 To create a chatbot using prompt templates, you will need to set up a chat API and then use the provided prompt template to generate structured responses. You can find an example of a chat prompt template here and further information on how to use it in the linked documentation.


In [16]:
def print_similarity_search(results):
    pass



In [17]:
results = vectordb.similarity_search(query=question, k=5)
for r in results:
    print(r.metadata['url'])
    print(r.page_content)

https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/custom_prompt_template.html
of default prompt templates here . Creating a Custom Prompt Template # There are essentially two distinct prompt templates available - string prompt templates and chat prompt templates. String prompt templates provides a simple prompt in string format, while chat prompt templates produces a more structured prompt to be used with a chat API. In this guide, we will create a custom prompt using a
https://python.langchain.com/en/latest/modules/prompts.html
retrying if necessary). Go Deeper # Prompt Templates Chat Prompt Template Example Selectors Output Parsers
https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/figma.html
have no idea if the Jon Carmack thing makes for better code. YMMV. # See https://python.langchain.com/en/latest/modules/models/chat/getting_started.html for chat info system_prompt_template = """You are expert coder Jon Carmack. Use th

In [18]:
vectordb.similarity_search(query=base_answer, k=5)

[Document(page_content='of default prompt templates here . Creating a Custom Prompt Template # There are essentially two distinct prompt templates available - string prompt templates and chat prompt templates. String prompt templates provides a simple prompt in string format, while chat prompt templates produces a more structured prompt to be used with a chat API. In this guide, we will create a custom prompt using a', metadata={'url': 'https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/custom_prompt_template.html'}),
 Document(page_content='PromptLayer ChatOpenAI # This example showcases how to connect to PromptLayer to start recording your ChatOpenAI requests. Install PromptLayer # The promptlayer package is required to use PromptLayer with OpenAI. Install promptlayer using pip. pip install promptlayer Imports # import os from langchain.chat_models import PromptLayerChatOpenAI from langchain.schema import HumanMessage Set the', metadata={'url': 'https://

In [19]:
vectordb.similarity_search(query=context_answer, k=5)

[Document(page_content='of default prompt templates here . Creating a Custom Prompt Template # There are essentially two distinct prompt templates available - string prompt templates and chat prompt templates. String prompt templates provides a simple prompt in string format, while chat prompt templates produces a more structured prompt to be used with a chat API. In this guide, we will create a custom prompt using a', metadata={'url': 'https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/custom_prompt_template.html'}),
 Document(page_content='to make constructing and working with prompts easy.\n\nThis section of documentation is split into four sections:\n\n**LLM Prompt Templates**\n\nHow to use PromptTemplates to prompt Language Models.\n\n**Chat Prompt Templates**\n\nHow to use PromptTemplates to prompt Chat Models.\n\n**Example Selectors**\n\nOften times it is useful to include examples in prompts.\nThese examples can be hardcoded, but it is often', meta

In [ ]:
question = "What is the map_reduce chain type?"
print("Base-Model:")
base_answer = llm(prompt=f"Question: {question}? Answer: ")
print(base_answer)
print("Context-Model:")
context_answer = qa.run(question)
print(context_answer)

# Appendix

In [20]:
from bs4 import BeautifulSoup
import numpy as np
import chromadb

client = chromadb.Client()

collection = client.create_collection('sample_collection')

documents = []
metadatas = []

for url, chunk in langchain_doc_chunks:
        documents.append(chunk)
        metadatas.append({'url': url})

collection.add(
    documents=documents, # we embed for you, or bring your own
    metadatas=metadatas, # filter on arbitrary metadata!
    ids=[str(x) for x in range(len(documents))], # must be unique for each doc 
)

print(np.mean([len(x) for x in documents]))
print(np.max([len(x) for x in documents]))

394.82080485952923
400


In [21]:
print(documents[0:10])

['404 Not Found\n | Read the Docs Read the Docs 404 Not Found You are browsing the documentation of langchain . The documentation version you are looking for at /en/deployments.html was not found. Documentation changes over time and pages are moved around. You can try to navigate to the index page of the project and use its navigation, or search for a similar page. Try searching? Search all the docs', 'Search all the docs Are you the project owner? Here are some tips to address 404 errors: Use your own custom 404 page: Read more » Create redirects when you move contents: Read more » Stay Updated Blog Sign up for our newsletter to get our latest blog updates delivered to your inbox weekly. Email Subscribe Loading... Thank you! You have successfully joined our subscriber list. Resources Tutorial', 'Resources Tutorial Team Documentation Going Ad-free Site Support Site Status Company Jobs Advertise with Us Read the Docs for Business Branding & Media Kit Privacy Policy Terms of Service © Co

In [22]:
results = collection.query(
    query_texts="What is a document loader?",
    n_results=5,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)
print(len(results['documents'][0]))

5


In [23]:
results

{'ids': [['192', '183', '187', '222', '182']],
 'embeddings': None,
 'documents': [['and explore other prompts, chains, and agents. Glossary : A glossary of all related terms, papers, methods, etc. Whether implemented in LangChain or not! Gallery : A collection of our favorite projects that use LangChain. Useful for finding inspiration or seeing how things were done in other applications. Deployments : A collection of instructions, code snippets, and template repositories for',
   'optimization, and prompt serialization. Memory : Memory is the concept of persisting state between calls of a chain/agent. LangChain provides a standard interface for memory, a collection of memory implementations, and examples of chains/agents that use memory. Indexes : Language models are often more powerful when combined with your own text data - this module covers best practices for doing',
   'each other or to events can be an interesting way to observe their long-term memory abilities. Personal Assista